#### Load Flower Dataset for Classification (pic)

-------------

In [46]:
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt

# import os
# import tensorflow as tf
# import tensorflow_datasets as tfds

In [47]:
# dataset, info = tfds.load('tf_flowers',  with_info=True, as_supervised=True)

In [48]:
# supervised_keys = {'image', 'label'}
# class_names = info.features['label'].names
# class_names

In [49]:
# splits={
#     'train':<SplitInfo num_examples=3670, num_shard=2>
# }

#### Download Flowers Data

---------------------

In [50]:
# for i, example in enumerate(dataset['train']):
#     image, label = example
#     save_dir = 'tf_flowers/{}'.format(class_names[label])
#     os.makedirs(save_dir, exist_ok=True)
#     filename = save_dir + "/" + str(i) + ".jpg"
#     tf.keras.preprocessing.image.save_img(filename, image.numpy())

#### Flowers Data Visualization

---------------

In [51]:
# data = dataset['train'].take(10)

# fix, ax = plt.subplots(2,5, figsize=(10,3))
# ax = ax.ravel()

# for i, example in enumerate(data):
#     image, label = example
#     image = image.numpy()

#     ax[i].imshow(image)
#     ax[i].set_title(class_names[label])
#     ax[i].axis('off')

# plt.tight_layout()

#### Preparing Data with Image Data Generator

---------------

In [52]:
# from tensorflow.keras.preprocessing.image import ImageDataGenerator
# from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
# from tensorflow.keras.models import Sequential

In [53]:
# datagen = ImageDataGenerator(rescale=1/255, validation_split=0.2)

# train_generator = datagen.flow_from_directory('tf_flowers', target_size=(224,224), batch_size=32, class_mode='categorical', subset='training')

# validation_generator = datagen.flow_from_directory('tf_flowers', target_size=(224,224), batch_size=32, class_mode='categorical', subset='validation' )

In [54]:
# # class names if we read from directory
# train_generator.class_indices

In [55]:
# # class names if we read data directly from tfds
# class_names

#### Baseline CNN Model Building

----------------------

In [56]:
# def get_base_model():
#     model = Sequential()
#     model.add(Conv2D(32, (3,3), activation='relu', input_shape=(224,224,3)))
#     model.add(MaxPooling2D(2,2))

#     model.add(Conv2D(64, (3,3), activation='relu'))
#     model.add(MaxPooling2D(2,2))

#     model.add(Conv2D(128, (3,3), activation='relu'))
#     model.add(MaxPooling2D(2,2))

#     model.add(Flatten())
#     model.add(Dense(512, activation='relu'))
#     model.add(Dense(5, activation='softmax'))

#     return model

# model = get_model()
    

In [57]:
# model.summary()

#### How to calculate Number of Parameters in CNN

-----------------------------

In [58]:
# output shapes
# CNN -> input_size-(filter_size-1)
# Pooling layer floor ((w-f)/s+1)
224-(3-1), (222-2)/2 + 1

(222, 111.0)

In [59]:
# Parameters
# (#input channel * filter_size*filter_size + 1)* #filters in current layer
(3*3*3+1)*32, (32*3*3+1)*64

(896, 18496)

In [60]:
# for dense layer
# (#prev cells + 1)* #cells in current layer
(86528+1)*512

44302848

#### Baseline CNN Model Training

-------------------------

In [61]:
# weights

In [62]:
# model.layers()

In [63]:
# weights, biases = model.layer[0].get_weights()
# len(biases)

In [64]:
# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# model.fit(train_generator, epochs=5, validation_data=validation_generator)

#### Train Model with TFDS Data without saving Locally part 1

-----------------

In [65]:
# split_data, dateset_info = tfds.load('tf_flowers',
#                                      split=['train[:80%]', 'train[:80%]'],
#                                      with_info=True, as_supervised=True)

In [66]:
# train_dataset, validation_dataset = split_data

In [67]:
# batch_size = 32
# image_sizez = (224,224)

# def preprocess(image, label):
#     image = tf.image.resize(image, image_size)
#     image = tf.cast(image, tf.float32)/255

#     return image, label

#### Train Model with TFDS Data without saving Locally part 2

--------------------

In [68]:
# train_dataset = train_dataset.map(preprocess)
# validation_dataset = validation_dataset.map(preprocess)

In [69]:
# train_dataset = train_dataset.batch(batch_size)
# validation_dataset = validation_dataset.batch(batch_size)

In [70]:
# model = get_model()

# model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# model.fit(train_dataset, epochs=5, validation_data = validation_dataset)

#### Import VGG16 from keras

---------------------------

In [71]:
# from tensorflow.keras.applications.vgg16 import VGG16

#### Data Augmentation for Training

---------------------

In [72]:
# datagen = ImageDataGenerator(rescale=1/255, validation_split=0.2, rotation_range=10,
#                              width_shift_range=0.1, height_shift_range=0.1,
#                              shear_range=0.1, zoom_range=0.1, horizontal_flip=True)

# train_generator=datagen.flow_from_directory('tf_flowers', target_size=(224,224),
#                                             batch_size=8, class_mode='categorical',
#                                             subset='training')

# validation_generator=datagen.flow_from_directory('tf_flowers', target_size=(224,224),
#                                                  batch_size=8, class_mode='categorical',
#                                                  subset='validation')

#### Make CNN Model with VGG16 Transfer Learning

--------------------------------------

In [73]:
# model_vgg16 = VGG16(input_shape=(224,224, 3), include_top=False, weights='imagenet')

In [74]:
# model_vgg16.summary()

In [75]:
# model_vgg16.layers

In [76]:
# for layer in model_vgg16.layers:
#     layer.trainable=False

In [77]:
# model_vgg16.summary()

In [78]:
# model = Sequential()
# model.add(model_vgg16)
# model.add(Flatten())
# model.add(Dense(512, activation='relu'))
# model.add(Dense(5, activation='softmax'))

In [79]:
# model.summary()

#### Model Training for Better Accuracy

------------------

In [80]:
# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# model.fit(train_generator, epochs=5, validation_data = validation_generator)

#### Train Any Model for Transfer Learning

-----------------------

In [81]:
# def train_model(new_model):
#     #freeze layers
#     for layer in new_model.layers:
#         layer.trainable=False

#     model = Sequential()
#     model.add(new_model)
#     model.add(Flatten())
#     model.add(Dense(512, activation='relu'))
#     model.add(Dense(5, activation='softmax'))

#     model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
#     model.fit(train_generator, epochs=5, validation_data = validation_generator)

In [82]:
# from tensorflow.keras.applications import InceptionV3

# new_model = InceptionV3(input_shape=(224,224,3), include_top=False, weights='imagenet')
# model = train_model(new_model)

In [83]:
# EfficientNetV2B3

#### Prediction, Save & Load Model Class Names


-------------------------------

In [84]:
# class_names = {v:k for k,v in train_generator.class_indices.items()}
# class_names

In [85]:
# import pickle as pkl

# data_model = {'class_names': ckass_names, 'model': model}
# pkl.dump(data_model, open('open.pkl', 'wb'))

In [86]:
# load_model = pkl.load('model.pkl', 'rb')

In [87]:
# load_model

In [88]:
# loaded_class_names = loaded_model['class_names']
# loaded_model = loaded_model['model']

#### Online Prediction of Flowers Classes

---------------------------------

In [89]:
# import requests
# from PIL import Image
# from io import BytesIO

In [91]:
# img_url = 'https://m.media-amazon.com/images/I/41rwKMqvKaL._SX300_SY300_QL70_ML2_.jpg'
# response = requests.get(img_url)
# img = Image.open(BytesIO(response.content)).resize((224,224))
## [img1, img2, img3]
# img = np.reshape(img, [1,224,224,3])
#img = img/255

# preds = loaded_model.predict(img)
# max_idx = np.argmax(pred[0])
# loaded_class_names[max_idx], preds[0][max_idx]*100